<a href="https://colab.research.google.com/github/bahulkark/Building-your-Deep-Neural-Network-Step-by-Step/blob/master/BDD_CODE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
## now we install TOG 
## to generate patched images.

!git clone https://github.com/git-disl/TOG
%cd ./TOG
# !pip install -r requirements.txt



Cloning into 'TOG'...
remote: Enumerating objects: 348, done.
remote: Counting objects: 100% (348/348), done.
remote: Compressing objects: 100% (297/297), done.
remote: Total 348 (delta 65), reused 290 (delta 35), pack-reused 0
Receiving objects: 100% (348/348), 42.76 MiB | 26.28 MiB/s, done.
Resolving deltas: 100% (65/65), done.
/content/TOG


In [ ]:
%cd ./TOG/
# %tensorflow_version 1.x
from attack_utils.patch_utils import extract_roi, extract_roi_multi, evaluate_vanishing_patch
from dataset_utils.preprocessing import letterbox_image_padded
from models.yolov3 import YOLOv3_Darknet53
from misc_utils.visualization import visualize_detections
from tensorflow import keras
import tensorflow.keras.backend as K
from tqdm import tqdm
from PIL import Image
import numpy as np
import datetime
import os
from tog.attacks import *

K.clear_session() 

weights = '/content/TOG/yolo_bdd.h5'  # TODO: Change this path to the victim model's weights
detector = YOLOv3_Darknet53(weights=weights)

In [5]:
## rm rf ?? 
# !rm -rf /content/TOG/assets
# !rm -rf /content/TOG/frcnn_utils
# !rm -rf /content/TOG/ssd_utils
# !rm ./*.ipynb

In [ ]:
## edit/future edit ??

# 1. edit patch_utils. for extract_roi asap 
# (DONE)2. edit visuallizations for new_vis = save function 



In [ ]:
## download scripts 


## kitti
# ## dld model 
# #https://drive.google.com/file/d/1--bzF2qygux4XVar4FWr4Aux9aIUt5Z-/view?usp=sharing
# wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1--bzF2qygux4XVar4FWr4Aux9aIUt5Z-' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1--bzF2qygux4XVar4FWr4Aux9aIUt5Z-" -O kitti_full.zip && rm -rf /tmp/cookies.txt
# #wget https://pjreddie.com/media/files/yolov3.weights
# unzip kitti_full.zip

# ## bdd
# ## dld model 
# #https://drive.google.com/file/d/1y8M7mEomuehHQBplDeMIe1pBZIxF59-w/view?usp=sharing
# !wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1y8M7mEomuehHQBplDeMIe1pBZIxF59-w' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1y8M7mEomuehHQBplDeMIe1pBZIxF59-w" -O yolo_bdd.h5 && rm -rf /tmp/cookies.txt

# ## dld model weights


# ## dld datasets. 
# #https://drive.google.com/file/d/1OOL4vcQNV34gQ_ZFt5oOM6KWij-AXRiq/view?usp=sharing
# !wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1OOL4vcQNV34gQ_ZFt5oOM6KWij-AXRiq' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1OOL4vcQNV34gQ_ZFt5oOM6KWij-AXRiq" -O bdd_processed.zip && rm -rf /tmp/cookies.txt


!unzip bdd_processed.zip

In [ ]:
!git clone https://github.com/AlexeyAB/darknet
%cd ./darknet
#Edit the makefile to enable gpu and opencv
!sed -i 's/GPU=0/GPU=1/g' Makefile
!sed -i 's/OPENCV=0/OPENCV=1/g' Makefile
#Make darknet
!make

In [ ]:
## fix this to exclude colab drives. 
!mkdir -p ./labels/val
!mkdir -p ./labels/train

import os 
import shutil
lst = os.listdir("/content/TOG/images/val")
for file in lst:
  if file[-3:] == "txt":
    shutil.copyfile("/content/TOG/images/val/"+file, "/content/TOG/darknet/labels/val/"+file)
    
import os 
import shutil
lst = os.listdir("/content/TOG/images/train")
for file in lst:
  if file[-3:] == "txt":
    shutil.copyfile("/content/TOG/images/train/"+file, "/content/TOG/darknet/labels/train/"+file)

%cd /content/TOG/images/train
!find . -name "*.txt" -type f -delete

%cd /content/TOG/images/val
!find . -name "*.txt" -type f -delete


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
## do that thing. 

ROOT_TRAIN = '/content/TOG/images/train'
ROOT_TEST = '/content/TOG/images/test'
ROOT_OUTPUT = '/content/drive/MyDrive/bdd_colab/pro_bdd_patchs/'
NUM_EPOCHS = 100
LR_INIT = 0.1
LR_MULTIPLIER = 0.1
BATCH_SIZE = 16
TOLERANCE_MAX = 5
TOLERANCE_DELTA = 0.01
SOURCE_CLASS = 'car'


## in decreasing order
PATCH_SIZES = [(64,64), (32,32), (14,14), (10,10), (6,6)]
patches = [np.full(shape=(1, *PATCH_SIZE, 3), fill_value=0.50) for _ in range(len(PATCH_SIZES))]
lr = [LR_INIT for _ in range(len(PATCH_SIZES))]


fpaths_train = [os.path.join(ROOT_TRAIN, fname) for fname in os.listdir(ROOT_TRAIN)
fpaths_test = [os.path.join(ROOT_TEST, fname) for fname in os.listdir(ROOT_TEST)]



In [13]:
print("training patchs: ", PATCH_SIZES)

NameError: ignored

In [ ]:
## Notice which rois to consider... 
## *** Current training: exclusive patch of size 6x6. 



for ip, patch in patches:
    print("training patch: ", PATCH_SIZES[ip])
    min_loss = np.float('inf')
    tolerance = 0

    output_folder = os.path.join(ROOT_OUTPUT, 'pro_vanishing_multi_{}'.format(PATCH_SIZES[ip]),
                                '%s_%s' % (datetime.datetime.now().strftime("%Y-%m-%d_%H:%M:%S"), SOURCE_CLASS))
    os.makedirs(output_folder)
    for epoch in range(NUM_EPOCHS):
        ####################################################################################################################
        # Training
        ####################################################################################################################
        epoch_loss = []
        batch_grad, batch_loss = [], []
        np.random.shuffle(fpaths_train)
        for fpath in tqdm(fpaths_train):
            # Preprocess input images
            input_img = Image.open(fpath)
            x_nat, x_bbox = letterbox_image_padded(input_img, size=detector.model_img_size)

            # Get roi candidates with an area higher than a predefined threshold to avoid trivial attacks
            detections_nat = detector.detect(x_nat, iou_threshold=.5, conf_threshold=.7)
            rois = extract_roi2(detections_nat, detector.classes.index(SOURCE_CLASS), (0, 0, 256, 416), min_size=MIN_ROI_SIZE, patch_size=PATCH_SIZE, input_img=x_nat[0, :, :, :], model_img_size=detector.model_img_size)
            if len(rois) == 0:
                continue

            # Apply adversarial patch to each of the rois
            x_adv = x_nat.copy()
            detections_target = detections_nat.copy()
            try:
                for _, _, (xmin, ymin, xmax, ymax), did in rois:
                    x_adv[:, ymin:ymax, xmin:xmax, :] = patch
            except Exception:
                pass
            # visualize_detections_basic2("./debug_out/"+str(ix), {'Original Image': (x_adv, detections_target, detector.model_img_size, detector.classes),
            #                     'patched_image': (x_adv, detections_target, detector.model_img_size, detector.classes)})


            # Compute gradients
            grad, loss = detector.compute_object_vanishing_gradient_and_loss(x_adv, detections=detections_target)

            # Clip gradients to the area where the adversarial patch is located
            grad = np.mean([grad[:, ymin:ymax, xmin:xmax, :] for _, _, (xmin, ymin, xmax, ymax), _ in rois], axis=0)
            batch_grad.append(grad)
            batch_loss.append(loss)

            if len(batch_loss) == BATCH_SIZE:  # Update the adversarial patch and log the loss over the mini-batch
                # print(patch.shape)
                # print(batch_grad[0].shape)
                patch = np.clip(patch - lr * np.mean(batch_grad, axis=0), 0.0, 1.0)
                epoch_loss.append(np.mean(batch_loss))
                batch_grad, batch_loss = [], []

        ####################################################################################################################
        # Testing
        ####################################################################################################################
        # Baseline = Random permutation of the adversarial patch (i.e., decorrelating pixels)
        patch_rand = np.reshape(patch.copy(), newshape=(patch.shape[0]*patch.shape[1]*patch.shape[2], patch.shape[3]))
        np.random.shuffle(patch_rand)
        patch_rand = np.reshape(patch_rand, newshape=patch.shape)
        num_rois, score_adv, score_rand = 0, 0, 0
        for fpath in fpaths_test:
            input_img = Image.open(fpath)
            x_nat, x_bbox = letterbox_image_padded(input_img, size=detector.model_img_size)

            # Get roi candidates with an area higher than a predefined threshold to avoid trivial attacks
            detections_nat = detector.detect(x_nat, iou_threshold=.5, conf_threshold=.7)
            rois = extract_roi2(detections_nat, detector.classes.index(SOURCE_CLASS), (0, 0, 256, 416), min_size=MIN_ROI_SIZE, patch_size=PATCH_SIZE, input_img=x_nat[0, :, :, :], model_img_size=detector.model_img_size)
            num_rois_x = len(rois)
            if num_rois_x == 0:
                continue

            x_adv, x_rand = x_nat.copy(), x_nat.copy()
            try: 
                for _, _, (xmin, ymin, xmax, ymax), _ in rois:
                    x_adv[:, ymin:ymax, xmin:xmax, :] = patch
                    x_rand[:, ymin:ymax, xmin:xmax, :] = patch_rand
            except Exception:
                pass
            detections_adv = detector.detect(x_adv, iou_threshold=.5, conf_threshold=.7)
            detections_rand = detector.detect(x_rand, iou_threshold=.5, conf_threshold=.7)
            # visualize_detections_basic2("./debug_out/test_"+str(ix), {'Original Image': (x_adv, detections_target, detector.model_img_size, detector.classes),
            #                     'patched_image': (x_adv, detections_target, detector.model_img_size, detector.classes)})


            score_adv_x, score_rand_x = evaluate_vanishing_patch(detector.classes.index(SOURCE_CLASS), rois, detections_adv, detections_rand)
            score_adv += score_adv_x
            score_rand += score_rand_x
            num_rois += num_rois_x

        # Compute training statistics
        epoch_loss = float(np.mean(epoch_loss))
        ASR_TOG = score_adv / num_rois
        ASR_Rand = score_rand / num_rois

        # Save the adversarial patch
        np.save(os.path.join(output_folder, 'exclusive_Epoch-%d_Loss-%.2f_ASR-%.2f.npy' % (epoch, epoch_loss, ASR_TOG)), patch)

        # Monitor training loss for learning rate scheduling
        if epoch_loss > min_loss - TOLERANCE_DELTA:
            tolerance += 1
            if tolerance >= TOLERANCE_MAX:
                lr[ip] *= LR_MULTIPLIER
                tolerance = 0
        else:
            tolerance = 0
        min_loss = min(min_loss, epoch_loss)

        # Print training progress
        print('[Epoch %d] LR: %f | Tol: %d/%d | Min. Loss: %.4f' % (epoch, lr, tolerance + 1, TOLERANCE_MAX, min_loss))
        print('  > Loss      : %.4f' % epoch_loss)
        print('  > ASR (TOG) : %d/%d = %.2f' % (score_adv, num_rois, ASR_TOG))
        print('  > ASR (Rand): %d/%d = %.2f' % (score_rand, num_rois, ASR_Rand))
        if (lr[ip] == 0.0):
            print("LR converged to 0.0. Stopping Training for patch {}".format(patches[ip]))

In [11]:
!rm -rf /content/TOG/json

In [14]:
import shutil
shutil.make_archive("/content/multipatchreal", "zip", "/content/TOG")

'/content/multipatchreal.zip'